In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from langchain_core.documents import Document

In [2]:
doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')
processed_content = doc['Processed_Content']

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\admin-npatel237\AppData\Local\Temp\ipykernel_1652\1769151165.py:1: SyntaxWarning: invalid escape sequence '\T'
  doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')


In [3]:
doc.columns

Index(['Section', 'Url', 'Content', 'Metadata', 'Processed_Content',
       'Processed_Section', 'Processed_Metadata'],
      dtype='object')

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="edwatanabe/bert-large-cased-pt-law")

No sentence-transformers model found with name edwatanabe/bert-large-cased-pt-law. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at edwatanabe/bert-large-cased-pt-law and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
result = embedding_model.embed_query(text=processed_content[67])

In [6]:
documents = [Document(page_content=row['Processed_Content'], metadata={'Section': row['Processed_Section'], 'Metadata': row['Processed_Metadata'],'Url': row['Url']}) for i, row in doc.iterrows()]

In [ ]:
embeddings = embeddings_model.embed_documents([doc.page_content for doc in documents])